In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import csv
import re
import io
import os

In [2]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None

In [4]:
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return 0
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return 0

In [5]:
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    best_score=0
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        syn1_2=[synset.path_similarity(ss) for ss in synsets2]
        if len(syn1_2)!=0:
            best = list([synset.path_similarity(ss) for ss in synsets2 if synset.path_similarity(ss) is not None])
            best_score=max(best) if best else 0
        else:
            best_score=0
 
        # Check that the similarity could have been computed
        if best_score is not 0:
            score += best_score
            count += 1
    
    # Average the values
    if count is not 0:
        score /= count
    return score

In [6]:
def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2 


In [7]:
#building similarity matrix
def find_similarity(reviews,similarity_matrix,size):
    similar_matrix=np.zeros((size,size))
    for i in range(size):
        for j in range(i):
            simi=symmetric_sentence_similarity(reviews.Text[i], reviews.Text[j])
            similar_matrix[i][j]=similar_matrix[j][i]=simi
    for i in range(size):
        similar_matrix[i][i]=1
    return similar_matrix

In [ ]:
review=pd.read_csv('processed_sentences.csv')
review.columns=['Text']
size=(review.shape[0])
print(size)
similarity_matrix=np.zeros((size,size))
similarity_matrix=find_similarity(review,similarity_matrix,size)
#file_name=file_name=os.path.splitext(a_file)[0]
#save_matrix(file_name,similarity_matrix)

111


In [ ]:
print(similarity_matrix)

In [ ]:
print(similarity_matrix.shape)

In [ ]:
similarity_matrix=pd.DataFrame(similarity_matrix)

In [ ]:
similarity_matrix.to_csv('similarity_matrix.csv')